In [34]:
import os
import sys
import re
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import doubletdetection

# Project level imports
from larval_gonad.notebook import Nb
from larval_gonad.io import cellranger_counts

In [2]:
# Setup notebook
nbconfig = Nb.setup_notebook(seurat_dir='../output/scrnaseq-wf/scrnaseq_combine_force')

last updated: 2018-10-29 
Git hash:


In [17]:
def _parse_bcs(fname):
    bcs = pd.read_csv(fname, sep='\t', index_col=0, header=None).index.str.extract('(\w+)-1').values
    return bcs.flatten()

In [47]:
def _dups_rep(n):
    fname = f'../output/scrnaseq-wf/scrnaseq_samples/testis{n}_force/outs/filtered_gene_bc_matrices/dm6.16/matrix.mtx'
    raw_counts = doubletdetection.load_mtx(fname)

    fname = f'../output/scrnaseq-wf/scrnaseq_samples/testis{n}_force/outs/filtered_gene_bc_matrices/dm6.16/barcodes.tsv'
    bcs = _parse_bcs(fname)

    zero_genes = np.sum(raw_counts, axis=0) == 0
    raw_counts = raw_counts[:, ~zero_genes]

    clf = doubletdetection.BoostClassifier(n_iters=50)
    doublets = clf.fit(raw_counts).predict(p_thresh=1e-7, voter_thresh=0.8)

    (doublets == 1).sum()

    putative_doublets = bcs[doublets == 1]
    
    dups =  [f'rep{n}_' + x for x in putative_doublets]
    
    with open(f'../output/notebook/2018-10-29_testing_doubletdetection_rep{n}.txt', 'w') as fh:
        fh.write('\n'.join(dups))

In [49]:
%%capture
r1 = _dups_rep(1)
r2 = _dups_rep(2)
r3 = _dups_rep(3)